In [5]:
import os
import sys
import queue
import platform

from PyQt5 import QtWidgets, QtGui, QtCore
import vlc


class MiniPlayer(QtWidgets.QMainWindow):
    """Stripped-down PyQt5-based media player class to sync with "master" video.
    """

    def __init__(self, master=None):
        QtWidgets.QMainWindow.__init__(self, master)

        # Create a basic vlc instance
        self.instance = vlc.Instance()

        self.media = None

        # Create an empty vlc media player
        self.mediaplayer = self.instance.media_player_new()

        self.init_ui()
        self.open_file()
       
        self.timer = QtCore.QTimer(self)
        self.timer.setInterval(10)
        self.timer.timeout.connect(self.play_mode)
       
        self.timer.start()
       
    def init_ui(self):
        """Set up the user interface
        """
        self.widget = QtWidgets.QWidget(self)
        self.setWindowFlags(QtCore.Qt.WindowStaysOnTopHint) ##상단 레이어
        self.setCentralWidget(self.widget)

        # In this widget, the video will be drawn
        if platform.system() == "Darwin":  # for MacOS
            self.videoframe = QtWidgets.QMacCocoaViewContainer(0)
        else:
            self.videoframe = QtWidgets.QFrame()

        self.palette = self.videoframe.palette()
        self.palette.setColor(QtGui.QPalette.Window, QtGui.QColor(0, 0, 0))
        self.videoframe.setPalette(self.palette)
        self.videoframe.setAutoFillBackground(True)

        self.vboxlayout = QtWidgets.QVBoxLayout()
        self.vboxlayout.addWidget(self.videoframe)
        self.widget.setLayout(self.vboxlayout)

    def open_file(self):
        """Open a media file in a MediaPlayer
        """
        filename = "한양대학교 ERICA 홍보영상.f248.webm"

        # getOpenFileName returns a tuple, so use only the actual file name
        self.media = self.instance.media_new(filename)
        # Put the media in the media player
        self.mediaplayer.set_media(self.media)

        # Parse the metadata of the file
        self.media.parse()
        # Set the title of the track as the window title
        self.setWindowTitle("{}".format(self.media.get_meta(0)))

        # The media player has to be 'connected' to the QFrame (otherwise the
        # video would be displayed in it's own window). This is platform
        # specific, so we must give the ID of the QFrame (or similar object) to
        # vlc. Different platforms have different functions for this
        if platform.system() == "Linux":  # for Linux using the X Server
            self.mediaplayer.set_xwindow(int(self.videoframe.winId()))
        elif platform.system() == "Windows":  # for Windows
            self.mediaplayer.set_hwnd(int(self.videoframe.winId()))
        elif platform.system() == "Darwin":  # for MacOS
            self.mediaplayer.set_nsobject(int(self.videoframe.winId()))

        # Start playing the video as soon as it loads
        self.mediaplayer.play()
       
    def play_mode(self):
        if self.mediaplayer.get_state() == vlc.State.Ended:
            # 영상이 다 재생되었을 때, 다시 처음으로 돌아가도록 합니다.
            self.mediaplayer.stop()
            self.mediaplayer.set_position(0)
            self.mediaplayer.play()

def main():
    app = QtWidgets.QApplication(sys.argv)

    player = MiniPlayer()
    player.show()
    player.resize(480, 480)
   
    sys.exit(app.exec_())


if __name__ == "__main__":
    main()

SystemExit: 0